<a href="https://colab.research.google.com/github/aleks-haksly/ClickHouse/blob/main/practice/part_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
!pip install clickhouse_driver --quiet
from clickhouse_driver import Client
from google.colab import userdata
import ipywidgets as widgets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 935.1/935.1 kB 6.3 MB/s eta 0:00:00


In [3]:
# connection requisites
host = 'clickhouse.lab.karpov.courses'
dbase = 'default'
port = 9000
user = 'student'
password = userdata.get('ch2020_pwd')

In [4]:
client = Client(host=host, port=port, user=user,
                password=password, database=dbase)

## Airbnb Data Base

### Задание 4.1
Посчитайте среднее расстояние до центра города и выведите идентификаторы объявлений о сдаче отдельных комнат, для которых расстояние оказалось меньше среднего. Результат отсортируйте по убыванию, тем самым выбрав комнату, которая является наиболее удаленной от центра, но при этом расположена ближе, чем остальные комнаты в среднем.

In [5]:
sql = f"""
WITH avg_room_distance as (
SELECT
  avg(geoDistance(13.4050, 52.5200, toFloat64OrNull(longitude), toFloat64OrNull(latitude))) AS avg_distance
FROM
  listings
WHERE
  room_type = 'Private room')

SELECT
  id,
  geoDistance(13.4050, 52.5200, toFloat64OrNull(longitude), toFloat64OrNull(latitude)) as distance
FROM
  listings
WHERE
  room_type = 'Private room' and distance < (SELECT avg_distance FROM avg_room_distance)
ORDER BY distance DESC
LIMIT 1
"""
result, columns = client.execute(sql, with_column_types=True)
pd.DataFrame(result, columns=[tuple[0] for tuple in columns])

,id,distance
0,22306678,4608.981934
